# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує)
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор.

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [11]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

load_dotenv(r"C:\Users\a.nemogushcha\OneDrive - ТОВ Смарт Дистрибюшн\Documents\SQL\tvaya\class.env")

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

assert all([DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME]), "Не всі змінні з .env зчитані"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")


In [12]:
tables = pd.read_sql("SHOW TABLES;", con=engine)
tables.head()

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders


In [20]:
def update_customer(customer_id: int, phone: str=None, city: str=None, addressLine1: str=None):
  
    set_parts = []
    params = {"id": customer_id}

    if phone is not None:
        set_parts.append("phone = :phone")
        params["phone"] = phone

    if city is not None:
        set_parts.append("city = :city")
        params["city"] = city

    if addressLine1 is not None:
        set_parts.append("addressLine1 = :addressLine1")
        params["addressLine1"] = addressLine1

    if not set_parts:
        raise ValueError("Не передано жодного поля для оновлення")

    sql = text(f"""
        UPDATE customers
        SET {", ".join(set_parts)}
        WHERE customerNumber = :id
    """)

    with engine.begin() as conn:
        conn.execute(sql, params)


In [21]:

before = pd.read_sql(
    "SELECT customerNumber, customerName, phone, city, addressLine1 FROM customers WHERE customerNumber = 103;",
    con=engine
)
before


,customerNumber,customerName,phone,city,addressLine1
0,103,Atelier graphique,555-1234,Kyiv,Main Ave 10


In [22]:
update_customer(customer_id=103, phone="555-1234", city="Kyiv", addressLine1="Main Ave 10")


after = pd.read_sql(
    "SELECT customerNumber, customerName, phone, city, addressLine1 FROM customers WHERE customerNumber = 103;",
    con=engine
)
after


,customerNumber,customerName,phone,city,addressLine1
0,103,Atelier graphique,555-1234,Kyiv,Main Ave 10


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [23]:
from sqlalchemy import text
from datetime import date, timedelta


In [24]:
def next_order_number(conn):
    q = text("SELECT COALESCE(MAX(orderNumber), 0) + 1 AS next_id FROM orders;")
    return conn.execute(q).scalar()


In [30]:
def create_order(customer_number: int, items, 
                 order_date=None, required_date=None, status="In Process", comments=None):
    
    if order_date is None:
        order_date = date.today()
    if required_date is None:
        required_date = order_date + timedelta(days=14)

    if not items:
        raise ValueError("Список items пуст")

    with engine.begin() as conn:  # begin => commit при успехе, rollback при исключении
  
        order_number = next_order_number(conn)

 
        insert_order = text("""
            INSERT INTO orders (orderNumber, orderDate, requiredDate, status, comments, customerNumber)
            VALUES (:orderNumber, :orderDate, :requiredDate, :status, :comments, :customerNumber)
        """)
        conn.execute(insert_order, dict(
            orderNumber=order_number,
            orderDate=order_date,
            requiredDate=required_date,
            status=status,
            comments=comments,
            customerNumber=customer_number
        ))

   
        insert_detail = text("""
            INSERT INTO orderdetails
                (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
            VALUES (:orderNumber, :productCode, :quantityOrdered, :priceEach, :orderLineNumber)
        """)
   
        select_product_for_update = text("""
            SELECT productCode, quantityInStock, MSRP
            FROM products
            WHERE productCode = :pc
            FOR UPDATE
        """)
        update_stock = text("""
            UPDATE products
            SET quantityInStock = quantityInStock - :qty
            WHERE productCode = :pc
        """)

        line_no = 1
        for it in items:
            pc = it["productCode"]
            qty = int(it["quantity"])

     
            row = conn.execute(select_product_for_update, {"pc": pc}).mappings().first()
            if row is None:
                raise ValueError(f"productCode {pc} не найден")

            if row["quantityInStock"] < qty:
                raise ValueError(f"Недостаточно запаса для {pc}: на складе {row['quantityInStock']}, нужно {qty}")

            price_each = row["MSRP"]

           
            conn.execute(insert_detail, dict(
                orderNumber=order_number,
                productCode=pc,
                quantityOrdered=qty,
                priceEach=price_each,
                orderLineNumber=line_no
            ))
            line_no += 1

        
            conn.execute(update_stock, {"qty": qty, "pc": pc})

        return order_number


In [35]:

new_order = create_order(
    customer_number=103,
    items=[
        {"productCode": "S10_1678", "quantity": 3},
        {"productCode": "S18_2248", "quantity": 2},
    ],
    comments="New online order"
)
print("Замовлення створено:", new_order)



Замовлення створено: 10428


In [32]:
hdr = pd.read_sql(
    f"SELECT orderNumber, orderDate, requiredDate, status, customerNumber FROM orders WHERE orderNumber = {new_order};",
    con=engine
)
display(hdr)



,orderNumber,orderDate,requiredDate,status,customerNumber
0,10427,2025-08-08,2025-08-22,In Process,103


In [33]:
det = pd.read_sql(f"""
    SELECT od.orderNumber, od.productCode, p.productName, od.quantityOrdered, od.priceEach, od.orderLineNumber
    FROM orderdetails od
    JOIN products p USING (productCode)
    WHERE od.orderNumber = {new_order}
    ORDER BY od.orderLineNumber
""", con=engine)
display(det)


,orderNumber,productCode,productName,quantityOrdered,priceEach,orderLineNumber
0,10427,S10_1678,1969 Harley Davidson Ultimate Chopper,3,95.70,1
1,10427,S18_2248,1911 Ford Town Car,2,60.54,2


In [34]:
stock = pd.read_sql("""
    SELECT productCode, productName, quantityInStock
    FROM products
    WHERE productCode IN ('S10_1678','S18_2248')
""", con=engine)
display(stock)

,productCode,productName,quantityInStock
0,S10_1678,1969 Harley Davidson Ultimate Chopper,7927
1,S18_2248,1911 Ford Town Car,536
